In [1]:
from wmts_extractor.extractor import Extractor
from wmts_extractor.utils import obtain_decision_table, filter_tiles_by_pr_table, OpenStreetMap

# Arguments

Dictionary with options supported by the `Extractor` module.

In [2]:
args = dict(
    start_datetime=None, # Ignore imagery acquired before start datetime – format DD/MM/YYYY HH:MM:SS
    end_datetime=None,   # Ignore imagery acquired after end datetime – format DD/MM/YYYY HH:MM:SS
    max_cloud=None,      # Max cloud cover in percentage - example: 0.5
    features=None,       # Identify imagery for download by specifying list unique feature identifiers defined in metadata
    overlap=None,        # Minimum percentage overlap - example: 80
    platforms=None,      # List of space-separated platforms - example: ['WorldView-01', 'GeoEye-01']
    max_resolution=None, # Max resolution in meters - example: 0.4
    period_resolution=None, # File location containing weights assigned to a pair time period/resolution
    overwrite=False,     # Overwrite existing output files – otherwise skip download
    max_downloads=None,  # Max compliant downloads for aoi
    dirs=None,           # Path structure
    format='COG',      # Output image format
    options='COMPRESS=LZW', # Output image creation options
    out_path='/home/images/', # specifies root directory to copy output images
    zoom=18,             # integer value between 1 and maximum zoom level supported by WMTS end point
)

# Configuration

The `config` dictionary specifies identity of Digital Globe WMTS end point along with necessary login 
credentials. It also defines full pathname to OGR-supported file defining one or more area of interest geometries. An example of template configuration dictionary is shown below:

In [3]:
config = dict(
    endpoint=dict(
        name='securewatch',
        credentials=dict(
            username='',
            password=''
        ),
        id='0f96fb54-cbb7-4191-afec-ea6372b311ab', # unique 32-character id assigned by Maxar to access services
        uri='https://securewatch.digitalglobe.com/earthservice/wmtsaccess',
        type='slippy',
        layer='DigitalGlobe:ImageryTileService', 
        format='png', 
        tilematrixset='EPSG:3857', 
        profile='Currency_RGB_Profile'    
    ),
    aoi=dict(
        pathname='/cfg/aoi/aoi_example.shp', # full pathname to OGR supported file encoding one or more Point, Line or Polygon geometries
        name='name'
    )
)

In [4]:
# Create the SecureWatch Extractor object
securewatch = Extractor(config, args)

# Get all tiles under the conditions given above
tiles = securewatch.get_tiles()

# Show the tiles table
tiles

,platform,uid,product,acq_datetime,cloud_cover,resolution,geometry,overlap,aoi_name
0,WorldView-02,0dc964c036d72bf62989c344b73aea63,Pan Sharpened Natural Color,2021-10-23 07:48:31,0.000000,0.4,"POLYGON ((43.64604 16.85920, 43.64606 17.03260...",100.0,SD_01.02
1,WorldView-01,551c0f28c37e426782c210051106fec9,Panchromatic,2021-10-22 10:33:51,0.000000,0.5,"POLYGON ((43.64130 16.82559, 43.64136 17.01903...",100.0,SD_01.02
2,WorldView-02,29a5e0a767f3c36434a185a7f1240337,Pan Sharpened Natural Color,2021-10-18 07:32:46,0.000000,0.5,"POLYGON ((43.67021 16.83705, 43.67033 17.02530...",100.0,SD_01.02
3,WorldView-02,30e73c0df4f44a7cafcd350cd72a0967,Pan Sharpened Natural Color,2021-10-12 07:54:17,0.000000,0.5,"POLYGON ((43.66862 17.20856, 43.76173 17.21923...",100.0,SD_01.02
4,WorldView-03,34af591aa9d53b742aaa718485a66378,Pan Sharpened Natural Color,2021-10-11 07:35:44,0.000000,0.3,"POLYGON ((43.71227 17.20761, 43.78169 17.20823...",100.0,SD_01.02
...,...,...,...,...,...,...,...,...,...
7177,WorldView-02,e56326008413597d676b2eec06c2412f,Pan Sharpened Natural Color,2017-07-17 07:38:44,0.000000,0.5,"POLYGON ((43.57956 17.01826, 43.58087 15.96352...",100.0,SD_01.05
7178,WorldView-03,06dc43142ceb07c1a7c06d6abe5db5de,Pan Sharpened Natural Color,2016-09-16 08:09:52,0.173242,0.3,"POLYGON ((43.69411 16.79939, 43.69411 16.79830...",100.0,SD_01.05
7179,WorldView-02,2750e3a11215256fa73451b4b8f1eab4,Pan Sharpened Natural Color,2013-12-04 08:15:17,0.000000,0.5,"POLYGON ((43.68452 15.95157, 43.68458 15.98425...",100.0,SD_01.05
7180,WorldView-02,18b7e9929d3bf12a6613051f2b5d847f,Pan Sharpened Natural Color,2013-02-14 08:07:26,0.000000,0.4,"POLYGON ((43.72873 16.54110, 43.72883 16.57299...",100.0,SD_01.05


In [5]:
filtered_tiles = securewatch.filter_tiles(tiles)
filtered_tiles

,platform,uid,product,acq_datetime,cloud_cover,resolution,geometry,overlap,aoi_name
0,WorldView-02,0dc964c036d72bf62989c344b73aea63,Pan Sharpened Natural Color,2021-10-23 07:48:31,0.000000,0.4,"POLYGON ((43.64604 16.85920, 43.64606 17.03260...",100.0,SD_01.02
1,WorldView-01,551c0f28c37e426782c210051106fec9,Panchromatic,2021-10-22 10:33:51,0.000000,0.5,"POLYGON ((43.64130 16.82559, 43.64136 17.01903...",100.0,SD_01.02
2,WorldView-02,29a5e0a767f3c36434a185a7f1240337,Pan Sharpened Natural Color,2021-10-18 07:32:46,0.000000,0.5,"POLYGON ((43.67021 16.83705, 43.67033 17.02530...",100.0,SD_01.02
3,WorldView-02,30e73c0df4f44a7cafcd350cd72a0967,Pan Sharpened Natural Color,2021-10-12 07:54:17,0.000000,0.5,"POLYGON ((43.66862 17.20856, 43.76173 17.21923...",100.0,SD_01.02
4,WorldView-03,34af591aa9d53b742aaa718485a66378,Pan Sharpened Natural Color,2021-10-11 07:35:44,0.000000,0.3,"POLYGON ((43.71227 17.20761, 43.78169 17.20823...",100.0,SD_01.02
...,...,...,...,...,...,...,...,...,...
7177,WorldView-02,e56326008413597d676b2eec06c2412f,Pan Sharpened Natural Color,2017-07-17 07:38:44,0.000000,0.5,"POLYGON ((43.57956 17.01826, 43.58087 15.96352...",100.0,SD_01.05
7178,WorldView-03,06dc43142ceb07c1a7c06d6abe5db5de,Pan Sharpened Natural Color,2016-09-16 08:09:52,0.173242,0.3,"POLYGON ((43.69411 16.79939, 43.69411 16.79830...",100.0,SD_01.05
7179,WorldView-02,2750e3a11215256fa73451b4b8f1eab4,Pan Sharpened Natural Color,2013-12-04 08:15:17,0.000000,0.5,"POLYGON ((43.68452 15.95157, 43.68458 15.98425...",100.0,SD_01.05
7180,WorldView-02,18b7e9929d3bf12a6613051f2b5d847f,Pan Sharpened Natural Color,2013-02-14 08:07:26,0.000000,0.4,"POLYGON ((43.72873 16.54110, 43.72883 16.57299...",100.0,SD_01.05


## Period Resolution filter

This filter has been added to fill a recurrent use case where the user needs a filtered list of products under a complex
criteria based on periods of time and resolution.

To do this, we have to design a bi-dimensional table that relates a pair of `time periods` and `resolution` values by 
what we call `weights`. An example of this table can be seen below.

If we pay attention to the first row, we can deduce that for a time period between `2011-01-01` and `2011-01-01` for the
given resolution values (`0.3`, `0.4` and `0.5`) there are some weights assigned with values `11`, `12` and `13`.

This filter contains the maximum number of images we want to obtain as additional condition, given by `number_images`.

**NOTE**: This filter can be used in addition to other filters such as minimum number for overlap (`-o`) and maximum
number of cloud cover (`-c`), but will ignore any other related to time periods (`-s`, `-e`) or resolution (`-r`).

In [6]:
period_resolution = {
    'number_images': 4,
    'resolutions': [0.3, 0.4, 0.5],
    'periods': [
        {
            'name': 'Period 1',
            'date_range': ['1/1/2011', '31/5/2014'],
            'weights': [[100, 90, 80], 
                        [80, 70, 50],
                        [80, 70, 50], 
                        [100, 90, 80]]
        },
        {
            'name': 'Period 2',
            'date_range': ['1/6/2015', '28/2/2019'],
            'weights': [[100, 80, 60],
                        [90, 70, 40],
                        [30, 20, 10],
                        [90, 70, 40],
                        [100, 80, 60]]
        },
        {
            'name': 'Period 3',
            'date_range': ['1/3/2020', '30/6/2021'],
            'weights': [[100, 90, 80],
                        [100, 90, 80]]
        }
    ]
}

obtain_decision_table(period_resolution)

,Period Name,0.3,0.4,0.5
2011-01-01 00:00:00,Period 1,100,90,80
2012-02-20 08:00:00,Period 1,80,70,50
2013-04-10 16:00:00,Period 1,80,70,50
2014-05-31 00:00:00,Period 1,100,90,80
2015-01-06 00:00:00,Period 2,100,80,60
2016-01-19 12:00:00,Period 2,90,70,40
2017-02-01 00:00:00,Period 2,30,20,10
2018-02-14 12:00:00,Period 2,90,70,40
2019-02-28 00:00:00,Period 2,100,80,60
2020-01-03 00:00:00,Period 3,100,90,80


In [7]:
final_tiles = filter_tiles_by_pr_table(tiles=filtered_tiles, 
                                       pr_table=obtain_decision_table(period_resolution), 
                                       number_images_per_period=4)

final_tiles.to_csv('/home/notebooks/final_tiles.csv')
final_tiles

,platform,uid,product,acq_datetime,cloud_cover,resolution,geometry,overlap,aoi_name,weights,period
0,WorldView-03,5c0eb667eaf5e64f72547232846c5b59,Pan Sharpened Natural Color,2020-10-10 07:58:24,0.001136,0.3,"POLYGON ((43.76302 16.82368, 43.76347 17.01675...",100.000000,SD_01.02,100.0,Period 3
1,WorldView-03,178c8fe8bf1166190fe523b67787f0c6,Pan Sharpened Natural Color,2020-12-30 07:34:51,0.000000,0.3,"POLYGON ((43.70606 17.20812, 43.77865 17.20783...",100.000000,SD_01.02,100.0,Period 3
2,WorldView-03,9e80dfa342be83140814f4a2ebaecb14,Pan Sharpened Natural Color,2021-01-18 07:33:27,0.000000,0.3,"POLYGON ((43.69367 17.20919, 43.77266 17.21480...",100.000000,SD_01.02,100.0,Period 3
3,WorldView-03,d1936319997a74f7b8b42a26dbad52e0,Pan Sharpened Natural Color,2020-10-29 07:57:52,0.004634,0.3,"POLYGON ((43.76513 16.83283, 43.76557 17.02132...",43.494497,SD_01.02,100.0,Period 3
4,WorldView-03,06dc43142ceb07c1a7c06d6abe5db5de,Pan Sharpened Natural Color,2016-09-16 08:09:52,0.173242,0.3,"POLYGON ((43.69411 16.79939, 43.69411 16.79830...",100.000000,SD_01.02,90.0,Period 2
...,...,...,...,...,...,...,...,...,...,...,...
950,WorldView-03,502091d4b7faffe28e8357c462342c68,Pan Sharpened Natural Color,2019-10-21 07:45:44,0.000000,0.4,"POLYGON ((43.67784 17.21045, 43.76493 17.21913...",100.000000,SD_01.04,80.0,Period 2
951,WorldView-01,77f647a90cbd9671919f0d64f632e64d,Panchromatic,2019-12-30 10:56:31,0.000000,0.5,"POLYGON ((43.68821 16.83317, 43.69015 17.03529...",100.000000,SD_01.04,60.0,Period 2
952,WorldView-02,a7ad09f58bb42908fc11509b023f9cca,Pan Sharpened Natural Color,2011-09-20 07:57:54,0.000000,0.4,"POLYGON ((43.70683 15.96328, 43.70685 15.99552...",100.000000,SD_01.04,90.0,Period 1
953,WorldView-02,18b7e9929d3bf12a6613051f2b5d847f,Pan Sharpened Natural Color,2013-02-14 08:07:26,0.000000,0.4,"POLYGON ((43.72873 16.54110, 43.72883 16.57299...",100.000000,SD_01.04,70.0,Period 1


In [ ]:
map = OpenStreetMap(df=final_tiles, popup_column='uid', epsg=3857, aoi_file=config.get('aoi').get('pathname'))
map.show()

In [ ]:
securewatch.push_tiles_to_s3(tiles=filtered_tiles[:2], s3_endpoint='https://s3-uk-1.sa-catapult.co.uk', s3_bucket='public-eo-data', s3_root_key='juan')

In [ ]:
# Download images to output folder given in args
#for file in securewatch.download_tiles(final_tiles):
#    print(f"Downloaded completed for {file}")